In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install openai

In [ ]:
import openai
openai.api_key = ""

Version1: get the sample directly. (Not used in the paper) The version used is titled with **Version 2 in the paper**

In [ ]:
import pandas as pd
import numpy as np
import os
import json
import time
adjdataset = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/adj.csv")

In [ ]:
print(adjdataset)

In [ ]:
gens1 = []
gens2 = []
cnt = 0
for i in range(len(adjdataset)):
    adj =adjdataset['adjective'][i]
    text1="We name as "
    text2=" 'adj'.Please give me an example noun 'entity1' that is adj and an example noun 'entity2' that is not adj. And give the comparable thing 'p' they can both do and could be used to judge whether something is 'adj' or not. Please give me a sentence 'entity1' is less 'p' than 'entity2'. 'adj' should not appear in that content. It may be confusing but just do it."
    q=text1+adj+text2
    prompt1 =q
    gens1.append(openai.chat.completions.create(
                  model="gpt-3.5-turbo-0301",
                  messages=[
                            {
                          "role": "user",
                          "content": prompt1,
                      }
                  ],

                  max_tokens=256,
                  temperature=0
    ))
    prompt2=q+"And please add some adjective for entity1 to let it more 'adj' and add some adjective for entity2 to let it less 'adj'."
    gens2.append(openai.chat.completions.create(
                  model="gpt-3.5-turbo-0301",
                  messages=[
                            {
                          "role": "user",
                          "content": prompt2,
                      }
                  ],

                  max_tokens=256,
                  temperature=0
    ))


    time.sleep(2)
    print("="*20)
    print(i, "\n")
    print(prompt1)
    print(gens1[i])
    print(prompt2)
    print(gens2[i])
    cnt += 1



print(cnt)


results1 = [gen.choices[0].message.content.strip() for gen in gens1]
results2 = [gen.choices[0].message.content.strip() for gen in gens2]

In [ ]:
np.save('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.0',np.array(results1))
np.save('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.1',np.array(results2))

In [ ]:
n=np.load('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.1.npy')
np.savetxt('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.1.txt',n, fmt='%s', newline='\n')

Version2: replace the sample.


In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/wsc_273_annotated_final.csv")

In [ ]:
import time
gens = []
answers = []
cnt = 0
for i in range(0, len(dataset)-1, 2):
    if i >= 20:
        break
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        # ans = json.loads(dataset["mod"][i])
        ans1, ans2 = ori_ans1, ori_ans2
    else:
        continue

    if isinstance(dataset["mod_text"][i], str):
        text1 = dataset["mod_text"][i]
        text2 = dataset["mod_text"][i+1]
    else:
        text1 = text1.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)
        text2 = text2.lower().replace(ori_ans1.lower(), ans1).replace(ori_ans2.lower(), ans2)

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"
    prompt = f"Q: Compare the two sentences and answer the questions\n1. {text1}\n2. {text2}\nWhat properties do these two sentences reflect about {ans1} and {ans2} in both sentences?\nPlease give two objects to replace the {ans1} and {ans2} according to the properties."
    print(prompt)


    gens.append(openai.chat.completions.create(
                  model="gpt-3.5-turbo-0301",
                  messages=[
                            {
                          "role": "user",
                          "content": prompt,
                      }
                  ],

                  max_tokens=256,
                  temperature=0
    ))
    print(gens)
    ssss
    answers.append([ans1, ans2])
    time.sleep(2)
    print("="*20)
    print(i, "\n")
    print(prompt)
    print(gens)
    cnt += 1


print(cnt)


results = [gen.choices[0].message.content.strip() for gen in gens]

In [ ]:
np.save('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.0',np.array(results))

In [ ]:
print(results[0])

In [ ]:
n=np.load('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.0.npy')
np.savetxt('/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/v1.0.txt',n, fmt='%s', newline='\n')

Analogous relation

In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/wsc_273_annotated_final.csv")

In [ ]:
import time
gens = []
answers = []
cnt = 0
for i in range(0, len(dataset)-1, 2):
    if i >= 245:
        i=i+1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans1, ans2 = ori_ans1, ori_ans2
    else:
        continue

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"





    prompt = f'''Compare the two sentences and answer the questions\n1.The bike passes the car because it is fast.\n2.The bike passes the car because it is slow.\nThink about the property these two sentences reflect about the bike and car in both sentences?\nPlease give two entities to replace the bike and car according to the properties.\n
In the two sentences, it reflects the speed of the bike and the car. In usual circumstances, the speed of a bike is slower than that of a car. Therefore, the two entities are:\n1. truck. 2. sports car.\nThe two sentences are\n1.The truck passes the sports car because it is fast.\n2.The truck passes the sports car because it is slow.\n\n
Compare the two sentences and answer the questions\n1.{text1}
2.{text2}
Think about the property these two sentences reflect about the {ans1} and {ans2} in both sentences?\nGive two entities with an analogous relation with the {ans1} and {ans2} according to the property.'''
    print(prompt)

#gpt-3.5-turbo-0125
#default: gpt-3.5-turbo-0301
    gens.append(openai.chat.completions.create(
                  model="gpt-3.5-turbo-0125",
                  messages=[
                            {
                          "role": "user",
                          "content": prompt,
                      }
                  ],

                  max_tokens=256,
                  temperature=0.5
    ))
    print(gens[cnt].choices[0].message.content.strip())
    answers.append([ans1, ans2])
    time.sleep(2)
    print("="*20)
    print(i, "\n")

    cnt += 1
    # if(i>5):
    #   break

print(cnt)


results = [gen.choices[0].message.content.strip() for gen in gens]

Compare the two sentences and answer the questions
1.The bike passes the car because it is fast.
2.The bike passes the car because it is slow.
Think about the property these two sentences reflect about the bike and car in both sentences?
Please give two entities to replace the bike and car according to the properties.

In the two sentences, it reflects the speed of the bike and the car. In usual circumstances, the speed of a bike is slower than that of a car. Therefore, the two entities are:
1. truck. 2. sports car.
The two sentences are
1.The truck passes the sports car because it is fast.
2.The truck passes the sports car because it is slow.


Compare the two sentences and answer the questions
1.The city councilmen refused the demonstrators a permit because they feared violence.
2.The city councilmen refused the demonstrators a permit because they advocated violence.
Think about the property these two sentences reflect about the The city councilmen and The demonstrators in both sente

In [ ]:
fn='replace_4_t_0.5_0125'
np.save(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/{fn}',np.array(results))

**Version 2 in the paper**

In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/wsc_273_annotated_final.csv")

In [ ]:
import time
gens = []
answers = []
cnt = 0
for i in range(0, len(dataset)-1, 2):
    if i >= 245:
        i=i+1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans1, ans2 = ori_ans1, ori_ans2
    else:
        continue

    def get_question(pronoun):
        pronoun = pronoun.lower()
        if pronoun in ["they", "them"]:
            return f" What do \"{pronoun}\" refer to?"
        if pronoun in ["he", "she", "it", "her", "him", "his"]:
            return f" What does \"{pronoun}\" refer to?"
    Prompt1 = f'''Compare the two sentences and answer the questions\n1.The bike passes the car because it is fast.\n2.The bike passes the car because it is slow.\nThink about the property these two sentences reflect about the bike and the car in both sentences.\nGive two entities with an analogous relation with the bike and the car according to the property.\n
In the two sentences, it reflects the speed of the bike and the car. In usual circumstances, the speed of a bike is slower than that of a car. Therefore, the two entities are:\n1. truck. 2. sports car.\nThe two sentences are\n1.The truck passes the sports car because it is fast.\n2.The truck passes the sports car because it is slow.\n\n
Compare the two sentences and answer the questions\n1.{text1}
2.{text2}
Think about the property these two sentences reflect about the {ans1} and {ans2} in both sentences.\nGive two entities with an analogous relation with the {ans1} and {ans2} according to the property.'''

    Prompt2 = f'''Compare the two sentences and answer the questions\n1.The ring doesn't fit into the handbag because it is too large.\n2.The ring doesn't fit into the handbag because it is too small.\nThink about the property these two sentences reflect about the ring and the handbag in both sentences.\nGive two entities with an analogous relation with the ring and the handbag according to the property.\n
In the two sentences, it reflects the size of the ring and the handbag. In usual circumstances, the size of a ring is smaller than that of a handbag. Therefore, the two entities are:\n1. pebble. 2. schoolbag.\nThe two sentences are\n1.The pebble doesn't fit into the schoolbag because it is too large.\n2.The pebble doesn't fit into the schoolbag because it is too small.\n\n
Compare the two sentences and answer the questions\n1.{text1}
2.{text2}
Think about the property these two sentences reflect about the {ans1} and {ans2} in both sentences.\nGive two entities with an analogous relation with the {ans1} and {ans2} according to the property.'''

    Prompt3 = f'''Compare the two sentences and answer the questions\n1.The body-builder doesn't lift the child because he is too heavy.\n2.The body-builder doesn't lift the child because he is too light.\nThink about the property these two sentences reflect about the body-builder and the child in both sentences.\nGive two entities with an analogous relation with the body-builder and the child according to the property.\n
In the two sentences, it reflects the weight of the body-builder and the child. In usual circumstances, the weight of a body-builder is larger than that of a child. Therefore, the two entities are:\n1. strong man. 2. little boy.\nThe two sentences are\n1.The strong man doesn't lift the little boy because he is too heavy.\n2.The strong man doesn't lift the little boy because he is too light.\n\n
Compare the two sentences and answer the questions\n1.{text1}
2.{text2}
Think about the property these two sentences reflect about the {ans1} and {ans2} in both sentences.\nGive two entities with an analogous relation with the {ans1} and {ans2} according to the property.'''

    Prompt4 = f'''Compare the two sentences and answer the questions\n1.The elite students were bullying the undisciplined students, so we punished them.\n2.The elite students were bullying the undisciplined students, so we rescued them.\nThink about the property these two sentences reflect about the elite students and the undisciplined students in both sentences.\nGive two entities with an analogous relation with the elite students and the undisciplined students according to the property.\n
In the two sentences, it reflects the compliance of the elite students and the undisciplined students. In usual circumstances, the compliance of the elite students is better than that of undisciplined students. Therefore, the two entities are:\n1. lawyers. 2. homeless guys.\nThe two sentences are\n1.The lawyers were bullying the homeless guys, so we punished them.\n2.The lawyers were bullying the homeless guys, so we rescued them.\n\n
Compare the two sentences and answer the questions\n1.{text1}
2.{text2}
Think about the property these two sentences reflect about the {ans1} and {ans2} in both sentences.\nGive two entities with an analogous relation with the {ans1} and {ans2} according to the property.'''


    Prompt5 = f'''Compare the two sentences and answer the questions\n1.The fish eats the worm, and it is tasty.\n2.The fish eats the worm, it is hungry.\nThink about the property these two sentences reflect about the fish and worm in both sentences.\nGive two entities with an analogous relation with the fish and worm according to the property.\n
In the two sentences, it reflects the taste of the fish and the worm. In usual circumstances, the taste of a fish is better than that of a worm. Therefore, the two entities are:\n1. ring-necked pheasant. 2. grasshopper.\nThe two sentences are\n1.The ring-necked pheasant eats the grasshopper, and it is tasty.\n2.The Ring-necked pheasant eats the grasshopper, it is hungry.\n\n
Compare the two sentences and answer the questions\n1.{text1}
2.{text2}
Think about the property these two sentences reflect about the {ans1} and {ans2} in both sentences.\nGive two entities with an analogous relation with the {ans1} and {ans2} according to the property.'''

    prompt=Prompt5

#gpt-3.5-turbo-0125
#default: gpt-3.5-turbo-0301
    gens.append(openai.chat.completions.create(
                  model="gpt-3.5-turbo-0125",
                  messages=[
                            {
                          "role": "user",
                          "content": prompt,
                      }
                  ],

                  max_tokens=256,
                  temperature=0.5
    ))
    print(cnt, "\n")
    print(prompt)
    print(gens[cnt].choices[0].message.content.strip())
    answers.append([ans1, ans2])
    time.sleep(1)
    print("="*20)

    cnt += 1


print(cnt)


results = [gen.choices[0].message.content.strip() for gen in gens]

0 

Compare the two sentences and answer the questions
1.The fish eats the worm, and it is tasty.
2.The fish eats the worm, it is hungry.
Think about the property these two sentences reflect about the fish and worm in both sentences.
Give two entities with an analogous relation with the fish and worm according to the property.

In the two sentences, it reflects the taste of the fish and the worm. In usual circumstances, the taste of a fish is better than that of a worm. Therefore, the two entities are:
1. ring-necked pheasant. 2. grasshopper.
The two sentences are
1.The ring-necked pheasant eats the grasshopper, and it is tasty.
2.The Ring-necked pheasant eats the grasshopper, it is hungry.


Compare the two sentences and answer the questions
1.The city councilmen refused the demonstrators a permit because they feared violence.
2.The city councilmen refused the demonstrators a permit because they advocated violence.
Think about the property these two sentences reflect about the The cit

In [ ]:
np.save(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/{fn}',np.array(results))

Data preprocessing

In [ ]:
text1=np.load(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/{fn}.npy")
text1
tt=[]
for i in text1:
  # Find the index of the first occurrence of "1."
  start_index = i.find("1.")

  # Find the index of the first occurrence of "." after "1."
  end_index = i.find(".", start_index+2)

  # Extract the content between "1." and "."
  print(start_index,end_index)
  content_between1 = i[start_index + len("1."):end_index].strip()
  start_index = i.find("2.")

  # Find the index of the first occurrence of "." after "1."
  end_index = i.find(".", start_index+2)

  # Extract the content between "1." and "."
  content_between2 = i[start_index + len("1."):end_index].strip()
  tt.append([i,content_between1,content_between2])

df = pd.DataFrame(tt,columns=['content','entity1','entity2'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/{fn}.csv', index=False)

276 300
168 179
232 242
383 409
134 144
197 210
193 203
259 267
261 271
421 487
248 277
224 250
307 332
209 244
325 376
263 274
269 290
134 144
166 194
320 336
339 365
181 191
131 155
138 155
268 354
173 190
258 282
247 256
311 332
255 262
281 292
215 226
232 294
268 280
244 253
123 136
297 307
378 410
242 257
241 250
320 341
153 180
138 164
294 303
201 207
292 338
146 158
150 162
399 448
312 352
191 204


In [ ]:
dataset = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/wsc_273_annotated_final.csv")
dataset1=pd.read_csv(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/{fn}.csv")

In [ ]:
#for generated ones
import time
import re

def replace_ignore_case(string, old_substring, new_substring):
    return re.sub(re.compile(re.escape(old_substring), re.IGNORECASE), new_substring, string)
gens = []
answers = []
cnt = 0
q1=[]
q2=[]
a1=[]
a2=[]


for i in range(0, len(dataset)-1, 2):
    if i >= 245:
        i=i+1
    text1 = dataset['text'][i]
    text2 = dataset['text'][i+1]
    pronoun = dataset['pronoun'][i]
    ori_ans1, ori_ans2 = json.loads(dataset["options"][i])
    if isinstance(dataset["mod"][i], str):
        ans1, ans2 = dataset1['entity1'][cnt], dataset1['entity2'][cnt]

        text1,text2= replace_ignore_case(text1,ori_ans1,ans1),replace_ignore_case(text2,ori_ans1,ans1)
        text1,text2= replace_ignore_case(text1,ori_ans2,ans2),replace_ignore_case(text2,ori_ans2,ans2)
        print(cnt)
        print(ans1,ans2)
        # print(text1)

    else:

        continue
    tt[cnt].append(text1)
    tt[cnt].append(text2)
    tt[cnt].append(ans1)
    tt[cnt].append(ans2)

    answers.append([ans1, ans2])

    cnt += 1


print(cnt)




In [ ]:
df = pd.DataFrame(tt,columns=['content','entity1','entity2','text1','text2','ans1','ans2'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/{fn}_final.csv', index=False)

In [ ]:
print(tt[1][1:5])

['elephant', 'backpack', "elephant doesn't fit into the brown suitcase because it is too large.", "elephant doesn't fit into the brown suitcase because it is too small."]


For the five prompts, put in one file

In [ ]:
dataset0=pd.read_csv(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/replace_temp_0.5_0125_0_final.csv")
dataset1=pd.read_csv(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/replace_temp_0.5_0125_1_final.csv")
dataset2=pd.read_csv(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/replace_temp_0.5_0125_2_final.csv")
dataset3=pd.read_csv(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/replace_temp_0.5_0125_3_final.csv")
dataset4=pd.read_csv(f"/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/replace_temp_0.5_0125_4_final.csv")
dataset=[dataset0,dataset1,dataset2,dataset3,dataset4]
dataset_t = pd.read_csv("/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/wsc_273_annotated_t.csv")

In [ ]:
print(dataset0)

                                              content  \
0   In the two sentences, it reflects the reason f...   
1   In the two sentences, it reflects the size of ...   
2   In the two sentences, it reflects the role of ...   
3   In the two sentences, it reflects the availabi...   
4   In the two sentences, it reflects the reluctan...   
5   In the two sentences, it reflects the speed of...   
6   In the two sentences, it reflects the strength...   
7   In the two sentences, it reflects the height o...   
8   In the two sentences, it reflects the start of...   
9   In the two sentences, it reflects the stabilit...   
10  In the two sentences, it reflects the role of ...   
11  In the two sentences, it reflects the emotiona...   
12  In the two sentences, it reflects the position...   
13  In the two sentences, it reflects Pete's feeli...   
14  In the two sentences, it reflects the actions ...   
15  In the two sentences, it reflects the volume o...   
16  In the two sentences, it re

In [ ]:
dd=[]
for i in range(len(dataset0['content'])):
  for j in range(5):

    dd.append([dataset[j]['content'][i],dataset[j]['text1'][i],dataset[j]['entity1'][i],dataset[j]['entity2'][i],dataset_t['pronoun'][2*i],dataset_t['label'][2*i]])
    dd.append([dataset[j]['content'][i],dataset[j]['text2'][i],dataset[j]['entity1'][i],dataset[j]['entity2'][i],dataset_t['pronoun'][2*i],dataset_t['label'][2*i+1]])



In [ ]:
df = pd.DataFrame(dd,columns=['content','text','entity','entity','pronoun','label'])
df.to_csv(f'/content/gdrive/MyDrive/long-tail_knowledge/wsc-test-master/generated_final.csv', index=False)